# 文本分类实例

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
import pandas as pd

In [7]:
data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [6]:
data = data.dropna()

In [8]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self):
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv").dropna()
    
    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [9]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


In [10]:
from torch.utils.data import random_split

train_set, valid_set = random_split(dataset, lengths=[0.9, 0.1])
len(train_set), len(valid_set)

(6989, 776)

In [34]:
import torch 

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [17]:
for i in range(10):
    print(train_set[i])

('住了美居.环境很好.服务很好,下次去一定还住~~~唯一不足的就是洗浴的时候水温不是很稳定.', 1)
('酒店位于南北主干道旁，交通非常便利，特别适合自驾车旅游者。（酒店门口免费停车）房间设施不错，服务态度挺好。就是牡丹节期间洛阳的酒店价格都上浮厉害。', 1)
('入住手续办理快速。入住时告知我所预定的豪华房没有房间，免费升级到公寓房。这里首先疑问一下，为什么入住前1天预定时携程系统显示房间状况是良好，第二天中午11点多入住时已经没有房间？是酒店的问题还是携程的问题？第二，我在前台的房间价目表上，并没有看见“公寓房”的牌价，于是很疑问的咨询前台，公寓房时什么性质的房间？回答是平时不对外的房间。没有办法只能入住公寓房，进房间一看，超级大，其实就是二房一厅，两个房间都有床，但是看看里面的设施物品，简直昏倒，最过分的是床单上面还有破洞，床铺铺设的根本不平整，连三星级的酒店都比不上。卫生间的地上很脏，及其不舒服。环视房间，我找不到一本服务手册，于是酒店电话说明、闭路电视选台介绍等等什么都不清楚。房间没有总控开关，没有门铃的免打扰、没有浴袍、没有针线包。。。连3星级酒店房间应该必备的物品，这里都没有！！！很气愤，酒店式公寓是没有星级评定的，我预定的是5星级酒店，却给我一个没有星级没有任何服务的公寓房，其实一个人住一个晚上，需要这样大的两室一厅的房间吗？关键是种种不便，我无法接受。既然是升级，应该升级为酒店有星级定义的房间！因为没有看见该酒店5星级的房间，因此只能打这样的分数，希望酒店以此为戒，也希望携程反思，为什么会出现这样的情况！', 0)
('房间价格奇贵，上网费2元每分钟，所有价格还要另加服务费。房间设施一般，电视信号时断时续。礼宾部最烂，免费的机场巴士好像是对客人的赏赐，坐了后觉得很不舒服。这家酒店我以后是绝对不会去了，其他华美达也都免了。', 0)
('很不错，服务和房间设施等都很好。我是端午节和女朋友一起去的，女朋友手机没电可问总台借万能充电器，很快就送来了。那的饭菜也很不错，好吃，呵呵。总的来说很好', 1)
('酒店风景蛮好的，服务也不错，去泸州的话是不错的选择', 1)
('还可以大堂也不错房间也不错价格也还合理免费的早餐也很不错。。比较满意下次还住的可能很大。。。', 1)
('订的单人间，结果没宽带，换了商务单人间，却是在后面的贵宾楼---一座四层

In [35]:
from sympy import true
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_func)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False, collate_fn=collate_func)

next(enumerate(train_loader))[1]

{'input_ids': tensor([[ 101, 6235, 2791,  ...,    0,    0,    0],
        [ 101,  122,  119,  ...,    0,    0,    0],
        [ 101, 4384, 1862,  ...,    0,    0,    0],
        ...,
        [ 101, 3025, 4923,  ..., 4172,  752,  102],
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 4384, 1862,  ..., 4638, 8024,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
        1, 0, 1, 1, 1, 0, 1, 1])}

In [44]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
optimizer = Adam(model.parameters(), lr=2e-5)

In [46]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in valid_loader:
            if torch.cuda.is_available():
                batch = {k : v.cuda() for k , v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(valid_set)
def train(epoch=5, log_step=100):

    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in train_loader:
            if torch.cuda.is_available():
                batch = {k : v.cuda() for k , v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")


In [47]:
train()

ep: 0, global_step: 0, loss: 0.8998633623123169
ep: 0, global_step: 100, loss: 0.228034108877182
ep: 0, global_step: 200, loss: 0.3832194209098816
ep: 0, acc: 0.8788659572601318
ep: 1, global_step: 300, loss: 0.34385472536087036
ep: 1, global_step: 400, loss: 0.31685203313827515
ep: 1, acc: 0.894329845905304
ep: 2, global_step: 500, loss: 0.11043170094490051
ep: 2, global_step: 600, loss: 0.274932324886322
ep: 2, acc: 0.8891752362251282
ep: 3, global_step: 700, loss: 0.07788784056901932
ep: 3, global_step: 800, loss: 0.22750771045684814
ep: 3, acc: 0.8891752362251282
ep: 4, global_step: 900, loss: 0.05651538074016571
ep: 4, global_step: 1000, loss: 0.010401512496173382
ep: 4, acc: 0.8956185579299927


In [53]:
sen = "我觉得这家酒店不好吃"
id2_label = {0:"差评!", 1:"好评!"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors='pt')
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label[pred.item()]}")


输入：我觉得这家酒店不好吃
模型预测结果:差评!


In [57]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

pipe(sen)

Device set to use cuda:0


[{'label': '差评!', 'score': 0.8165022730827332}]